In [6]:
pip install requests pandas geopandas shapely

In [7]:
from datetime import date, timedelta
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import os

In [8]:
def get_keycloak(username: str, password: str) -> str:
    print(f"Username: {username}, Password: {password}")  # Debugging statement
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Response from the server was: {r.json()}"
        )
    return r.json()["access_token"]

In [9]:
copernicus_user = "21ssds415008@msruas.ac.in"
copernicus_password = "Myresearch@2021"
data_collection = "SENTINEL-2"  # Sentinel satellite
ft='POLYGON ((57.692102513344906 -20.35226228118077, 57.692102513344906 -20.4745116407249, 57.77699025382094 -20.4745116407249, 57.77699025382094 -20.35226228118077, 57.692102513344906 -20.35226228118077))'
data_collection = "SENTINEL-2" # Sentinel satellite
today = date(2020,9,10)
today_string = today.strftime("%Y-%m-%d")
yesterday = today - timedelta(days=30)
yesterday_string = yesterday.strftime("%Y-%m-%d")

json_ = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{ft}') and ContentDate/Start gt {yesterday_string}T00:00:00.000Z and ContentDate/Start lt {today_string}T00:00:00.000Z&$count=True&$top=1000"
).json()

p = pd.DataFrame.from_dict(json_["value"])  # Fetch available dataset

In [10]:
if p.shape[0] > 0:
    p["geometry"] = p["GeoFootprint"].apply(shape)
    productDF = gpd.GeoDataFrame(p).set_geometry("geometry")  # Convert PD to GPD
    productDF = productDF[~productDF["Name"].str.contains("L1C")]  # Remove L1C dataset
    print(f"Total L2A tiles found {len(productDF)}")
    productDF["identifier"] = productDF["Name"].str.split(".").str[0]
    allfeat = len(productDF)

    if allfeat == 0:
        print("No tiles found for today")
    else:
        # Download all tiles from server
        for index, feat in enumerate(productDF.iterfeatures()):
            try:
                session = requests.Session()
                keycloak_token = get_keycloak(copernicus_user, copernicus_password)
                session.headers.update({"Authorization": f"Bearer {keycloak_token}"})
                url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({feat['properties']['Id']})/$value"
                response = session.get(url, allow_redirects=False)
                while response.status_code in (301, 302, 303, 307):
                    url = response.headers["Location"]
                    response = session.get(url, allow_redirects=False)
                print(feat["properties"]["Id"])
                file = session.get(url, verify=False, allow_redirects=True)

                with open(
                    f"{feat['properties']['identifier']}.zip",  # Location to save zip from Copernicus
                    "wb",
                ) as p:
                    print(feat["properties"]["Name"])
                    p.write(file.content)
            except Exception as e:
                print(f"Error downloading tile: {e}")
else:
    print("No data found")

Total L2A tiles found 12
Username: 21ssds415008@msruas.ac.in, Password: Myresearch@2021
10c899b1-3bcd-5e09-98b1-6839c7beccd8


KeyboardInterrupt: 